# Write outlook online meeting schedule to Google Calendar using Google Calendar API
## Steps:
* From Google Calendar get all events and store them into the event list.
* From Outlook get online meeting appointments which in specific term and store them into appointment list.
* Using EntryId, find an event in the event list. If it is found, update it and delete it from the event list. If it is not found, insert it.
* Delete all events that remain in the event list. They are deleted or out of term.

In [5]:
#from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import re
import win32com.client

In [6]:
import json
# load setting file(json)
with open("calendar.json", "r", encoding="utf-8") as f:
    dic = json.load(f)

# "id":"xxxxx@group.calendar.google.com"
calendar_id = dic["id"]

# "tools": {"zoom": "zoom.us","webex": "webex.com", "teams": "teams.microsoft.com"}
tools = dic["tools"]

if "omit" in dic:
    omit=dic["omit"]
else:
    omit=[]

if "weeks" in dic:
    weeks_ahead=dic["weeks"]
else:
    weeks_ahead=3

for k in tools:
    print(k,tools[k])
print("omit",omit)
print("weeks ahead",weeks_ahead)

zoom zoom.us
webex webex.com
teams teams.microsoft.com
案件MTG 案件MTG
omit ['DBSV-CPU']
weeks ahead 4


In [7]:
# Outlook appointment search term:From.
st = datetime.date.today() - datetime.timedelta(days=1)
# Outlook appointment search term:To.
ed = datetime.date.today() + datetime.timedelta(weeks=weeks_ahead)

# Google settingsレンダのやつ
SCOPES = ["https://www.googleapis.com/auth/calendar"]
cred_json = "credentials.json"

# Load token
creds = None
if os.path.exists("token.pickle"):
    with open("token.pickle", "rb") as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(cred_json, SCOPES)
        creds = flow.run_local_server(port=0)
    with open("token.pickle", "wb") as token:
        pickle.dump(creds, token)

# Create Google Calendar service
service = build("calendar", "v3", credentials=creds)

# Get all events from Google Calendar
print("Getting all events")
event_result = (
    service.events()
    .list(
        calendarId=calendar_id,
        #                                     timeMin=st.isoformat()+'T00:00:00Z',
        #                                     timeMax=ed.isoformat()+'T00:00:00Z',
        singleEvents=True,
        orderBy="startTime",
    )
    .execute()
)
events = event_result.get("items", [])

# Display all events
if not events:
    print("No upcoming events found.")
for i, event in enumerate(events):
    start = event["start"].get("dateTime", event["start"].get("date"))
    start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
    print("{:02}".format(i), start, event["summary"])

Getting all events
00 2021-06-22 09:00:00 zoom:重要案件審議会（地域会社（データ信越））
01 2021-06-22 09:30:00 案件MTG:案件MTG
02 2021-06-22 14:00:00 teams:FW: PM配置基準見直し(公社／技統本）
03 2021-06-22 15:00:00 teams:地域案件Ｔ支援Ｇ定例(毎火15時)
04 2021-06-23 13:00:00 teams:【時間帯変更（再）】村山GTB向け資料RV
05 2021-06-23 15:00:00 teams:PMO部課長会議
06 2021-06-24 13:30:00 teams:グループ会社案件担当定例MTG（地域定例）
07 2021-06-24 15:00:00 zoom:PMO全体会議
08 2021-06-25 11:00:00 zoom:6月期　重案審事前説明（G事業統括部長主査）
09 2021-06-25 13:00:00 zoom:【ND関西】竹中工務店
10 2021-06-28 13:00:00 zoom:重案審（G事業統括部長主査）
11 2021-06-28 13:30:00 zoom:【ND関西】中立電機　週次進捗会議
12 2021-06-29 09:30:00 案件MTG:案件MTG
13 2021-06-29 15:00:00 teams:地域案件Ｔ支援Ｇ定例(毎火15時)
14 2021-06-30 15:00:00 teams:PMO部課長会議
15 2021-06-30 17:30:00 zoom:NDI情報交換会
16 2021-07-01 13:30:00 teams:グループ会社案件担当定例MTG（地域定例）
17 2021-07-02 09:00:00 zoom:重案審（G事業統括部長主査）
18 2021-07-02 13:00:00 zoom:【ND関西】竹中工務店
19 2021-07-05 13:30:00 zoom:【ND関西】中立電機　週次進捗会議
20 2021-07-06 09:30:00 案件MTG:案件MTG
21 2021-07-06 15:00:00 teams:地域案件Ｔ支援Ｇ定例(毎火15時)
22 2021-07-07 15:00:00 t

In [8]:
# tools = {"zoom": "zoom.us", "webex": "webex.com", "teams": "teams.microsoft.com"}

# get an event that has specific entry-id from events and return its index.
def getEventIndex(events, item):
    for i, ev in enumerate(events):
        if item.EntryId in ev["description"]:
            start = ev["start"].get("dateTime", ev["start"].get("date"))
            # You need [timzone] to compare so I added .astimezone().
            # I don't know why but Outlook appointments are in UTC timezone 
            # and after .astimezone() they change to JST. So you have to change it to UTC.
            start = (
                datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
                .astimezone()
                .replace(tzinfo=datetime.timezone.utc)
            )
            td = start - item.Start

            if td == datetime.timedelta(0):
                return i
    return -1


# is the appointment onlie meeting?
def isOnline(item):
    for k, v in tools.items():
        if v in item.Body:
            for sub in omit:
                if sub in item.Subject:
                    return False, None
            return True, k
    return False, None


# objects for Outlook
app = win32com.client.Dispatch("Outlook.Application")
root = app.Session.DefaultStore.GetRootFolder()
ns = app.GetNamespace("MAPI")
cal = ns.GetDefaultFolder(9)

# filter string for specific term
filterStr = '[Start]>="{0}" and [Start]<"{1}"'.format(
    st.strftime("%Y/%m/%d"), ed.strftime("%Y/%m/%d")
)
print("filter", filterStr)

# Sort appointments and divide reccurcible ones.
appointments = cal.Items
appointments.sort("[Start]")
appointments.IncludeRecurrences = True
restricted = appointments.Restrict(filterStr)
print("from", st, "to", ed)
counter = 0

# Access to Outlook appointment
for item in restricted:
    # Read each item's body and find tools string in it. If they has, that's what you need
    ret, prefix = isOnline(item)
    countStr = "{:02}".format(counter)
    if ret:
        # Add pre-fix 
        summary = prefix + ":" + item.subject

        # Copy body part until you find password
        bd = item.body.splitlines()
        desc = ""
        for line in bd:
            desc += line + "\n"
            if "パスワード" in line:
                break
        desc += "EntryId={}".format(item.EntryId)
        # Build a JSON data for a google calendar event.
        event = {
            "summary": summary,
            "description": desc,
            "start": {
                "dateTime": item.Start.strftime("%Y-%m-%dT%H:%M:%S"),
                "timeZone": "Japan",
            },
            "end": {
                "dateTime": item.End.strftime("%Y-%m-%dT%H:%M:%S"),
                "timeZone": "Japan",
            },
        }

        # Check if the event already in envets list
        index = getEventIndex(events, item)

        if index < 0:
            # It's not found in list, so insert a new event
            print(countStr, "insert", item.Start.isoformat(), item.subject)
            service.events().insert(
                calendarId=calendar_id,
                body=event).execute()
        else:
            # It found in the list, so POP it to delete.
            ev = events.pop(index)
            updated = datetime.datetime.fromisoformat(
                ev["updated"].replace("Z", "+00:00")
            )
            modified = item.LastModificationTime + datetime.timedelta(hours=-9)
            # print(modified.isoformat(),updated)
            if updated < modified:
                # Update it
                print(countStr, "update", item.Start.isoformat(), item.subject)
                service.events().update(
                    calendarId=calendar_id, 
                    eventId=ev["id"], 
                    body=event
                ).execute()
            else:
                print(countStr, "no upd", item.Start.isoformat(), item.subject)
            start = ev["start"].get("dateTime", ev["start"].get("date"))
            start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
        counter += 1

# Delete all events remaining in the list
for event in events:
    print("delete", event["start"]["dateTime"], event["summary"])
    service.events().delete(
        calendarId=calendar_id, 
        eventId=event["id"]).execute()
print("done.")

filter [Start]>="2021/06/22" and [Start]<"2021/07/21"
from 2021-06-22 to 2021-07-21
00 no upd 2021-06-22T09:00:00+00:00 重要案件審議会（地域会社（データ信越））
01 no upd 2021-06-22T09:30:00+00:00 案件MTG
02 no upd 2021-06-22T14:00:00+00:00 FW: PM配置基準見直し(公社／技統本）
03 no upd 2021-06-22T15:00:00+00:00 地域案件Ｔ支援Ｇ定例(毎火15時)
04 no upd 2021-06-23T13:00:00+00:00 【時間帯変更（再）】村山GTB向け資料RV
05 no upd 2021-06-23T15:00:00+00:00 PMO部課長会議
06 no upd 2021-06-24T13:30:00+00:00 グループ会社案件担当定例MTG（地域定例）
07 no upd 2021-06-24T15:00:00+00:00 PMO全体会議
08 no upd 2021-06-25T11:00:00+00:00 6月期　重案審事前説明（G事業統括部長主査）
09 no upd 2021-06-25T13:00:00+00:00 【ND関西】竹中工務店
10 no upd 2021-06-28T13:00:00+00:00 重案審（G事業統括部長主査）
11 no upd 2021-06-28T13:30:00+00:00 【ND関西】中立電機　週次進捗会議
12 no upd 2021-06-29T09:30:00+00:00 案件MTG
13 no upd 2021-06-29T15:00:00+00:00 地域案件Ｔ支援Ｇ定例(毎火15時)
14 no upd 2021-06-30T15:00:00+00:00 PMO部課長会議
15 no upd 2021-06-30T17:30:00+00:00 NDI情報交換会
16 no upd 2021-07-01T13:30:00+00:00 グループ会社案件担当定例MTG（地域定例）
17 no upd 2021-07-02T09:00:00+00:00 重案審（G事業統